#### colab로 시작하기

<a href="https://colab.research.google.com/github/LDJWJ/AISTUDY/blob/master/DL05_03_RNN_v10" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Sequence to Sequence
  * 영어 단어를 한국어 단어로 변역하는 프로그램 만들어보기

* Seq2Seq는 구글의 기계번역에 사용되는 신경망 모델
* **입력**의 신경망의 **인코더**와 **출력**을 위한 신경망 **디코더**로 구성

In [1]:
import tensorflow as tf
import numpy as np

C:\Users\front\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz나무놀이소녀사랑음식아기카드']
num_dic = {n:i for i, n in enumerate(char_arr)} # 글자:숫자를 하나의 데이터 셋
print(num_dic)
dic_len = len(num_dic)    # 총 길이

{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '나': 29, '무': 30, '놀': 31, '이': 32, '소': 33, '녀': 34, '사': 35, '랑': 36, '음': 37, '식': 38, '아': 39, '기': 40, '카': 41, '드': 42}


In [3]:
seq_data = [['tree', '나무'], ['game', '놀이'], ['girl', '소녀'], 
            ['love', '사랑'], ['food', '음식'], ['baby', '아기'], 
            ['card', '카드'] ]

### 데이터 준비
 * 인코더의 입력값 : 입력단어 한글자씩 떼어 배열로 만들기 
 * 디코더의 입력값 : 출력단어의 글자들을 배열로 만들고, 시작을 나타내는 'S'을 붙임
 * 디코더의 출력값 : 디코더의 셀의 출력값을 만들고, 출력을 나타내는 'E'을 마지막에 붙임

In [4]:
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []

    for seq in seq_data:
        # 인코더 셀의 입력값. 입력단어의 글자들을 한글자씩 떼어 배열로 만든다.
        # seq_data의 영단어를 한글자씩(index 리스트)
        input = [num_dic[n] for n in seq[0]]
        # 디코더 셀의 입력값. 시작을 나타내는 S 심볼을 맨 앞에 붙여준다.
        # 한글의 단어에 앞'S'을 붙이고 한글자씩(index 얻기)
        output = [num_dic[n] for n in ('S' + seq[1])]
        # 학습을 위해 비교할 디코더 셀의 출력값. 끝나는 것을 알려주기 위해 마지막에 E 를 붙인다.
        # 한글의 단어에 맨뒤 'E'을 붙이고 한글자씩(index얻기)
        target = [num_dic[n] for n in (seq[1] + 'E')]

        input_batch.append(np.eye(dic_len)[input]) # 원핫(인코더 입력)
        output_batch.append(np.eye(dic_len)[output]) # 원핫(디코더 입력)
        
        # 출력값만 one-hot 인코딩이 아님 (sparse_softmax_cross_entropy_with_logits 사용)
        # 나E(20,1), 무E(30,1) - 디코더 출력
        target_batch.append(target)

    return input_batch, output_batch, target_batch

In [5]:
make_batch(['tree', '나무'])

([array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
  array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])],
 [array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
  array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.,

### 하이퍼 파라미터 설정
 * 학습률
 * 은닉층 노드수
 * 총 에폭수
 * 타깃 클래스, 입력 노드

In [6]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 100
 
n_class = n_input = dic_len

### 인코더의 입력값, 디코더의 입력값 
 * [batch_size, time steps, input_size] -> [데이터개수, 작업 스텝(인코더), 입력 크기]

### 디코더의 출력값
 * [batch_size, time steps] -> [배치 데이터개수, 작업 스텝(디코더)]
 
### 배치크기와 작업스텝은 입력때마다 다를 수 있다. (None으로 처리)

In [7]:
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

* 주의 : 같은 배치 데이터는 글자수(단계)는 같아야 함.
* dynamic_rnn의 옵션 sequence_length를 사용하면 길이가 다른 단어들도 한번에 입력이 가능함.
* 단, 길이가 다르더라도 짧은 단어는 가장 긴단어에 맞춰서 글자를 채워야 하므로 의미 없는 값인 'P'를 이용하여 부족한 글자수를 채움.

### 체크 : 두번 실행시 아래와 같은 에러 발생할 수 있음. (재실행)
 * Variable encode/rnn/basic_rnn_cell/kernel already exists, disallowed.

### 중요
 * decode를 설계할 때, 초기 상태를 인코더의 출력(enc_states)로 넣어야 함.
   * dynamic_rnn(.., initial_state=enc_states ..) 로 가능

In [8]:
# 인코더 셀을 구성한다.
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)   # 기본 셀 적용
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, 
                                    output_keep_prob = 0.5)  # Dropout 적용
    
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, 
                                            enc_input, 
                                            dtype=tf.float32)

# 디코더 셀을 구성한다.
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)  # 기본 셀 적용
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, 
                                             output_keep_prob=0.5)   # Dropout 적용
    
    # Seq2Seq 모델은 인코더 셀의 최종 상태값을
    # 디코더 셀의 초기 상태값으로 넣어주는 것이 핵심.
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, 
                                            dec_input, 
                                            initial_state=enc_states,  # *인코더의 출력(enc_state)을 넣어야
                                            dtype=tf.float32)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
model = tf.layers.dense(outputs, n_class, activation=None)

cost = tf.reduce_mean( 
           tf.nn.sparse_softmax_cross_entropy_with_logits(
              logits=model, labels=targets))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


### 학습 시키기 

In [12]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], 
                       feed_dict={enc_input:input_batch, 
                                  dec_input:output_batch,
                                  targets:target_batch} )
    
    print('Epoch:  {}'.format(epoch + 1),
          'cost = {}'.format(loss))

print("최적화 완료!")

Epoch:  1 cost = 3.779522180557251
Epoch:  2 cost = 2.905961275100708
Epoch:  3 cost = 1.7530995607376099
Epoch:  4 cost = 1.1898877620697021
Epoch:  5 cost = 0.7002539038658142
Epoch:  6 cost = 0.24918495118618011
Epoch:  7 cost = 0.25469326972961426
Epoch:  8 cost = 0.10096266120672226
Epoch:  9 cost = 0.08792263269424438
Epoch:  10 cost = 0.035645339637994766
Epoch:  11 cost = 0.07216156274080276
Epoch:  12 cost = 0.021181698888540268
Epoch:  13 cost = 0.03957994654774666
Epoch:  14 cost = 0.010124028660356998
Epoch:  15 cost = 0.010288801975548267
Epoch:  16 cost = 0.012970870360732079
Epoch:  17 cost = 0.003536701900884509
Epoch:  18 cost = 0.004682973027229309
Epoch:  19 cost = 0.00412184651941061
Epoch:  20 cost = 0.0031117626931518316
Epoch:  21 cost = 0.0037098596803843975
Epoch:  22 cost = 0.002357000019401312
Epoch:  23 cost = 0.0016109187854453921
Epoch:  24 cost = 0.0016084116650745273
Epoch:  25 cost = 0.0012046569027006626
Epoch:  26 cost = 0.0019429491367191076
Epoch:  

In [13]:
#########
# 번역 테스트
######
# 단어를 입력받아 번역 단어를 예측하고 디코딩하는 함수
def translate(word):
    # 이 모델은 입력값과 출력값 데이터로 [영어단어, 한글단어] 사용하지만,
    # 예측시에는 한글단어를 알지 못하므로, 디코더의 입출력값을 의미 없는 값인 P 값으로 채운다.
    # ['word', 'PPPP']
    seq_data = [word, 'P' * len(word)]

    input_batch, output_batch, target_batch = make_batch([seq_data])

    # 결과가 [batch size, time step, input] 으로 나오기 때문에,
    # 2번째 차원인 input 차원을 argmax 로 취해 가장 확률이 높은 글자를 예측 값으로 만든다.
    prediction = tf.argmax(model, 2)

    result = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})

    # 결과 값인 숫자의 인덱스에 해당하는 글자를 가져와 글자 배열을 만든다.
    decoded = [char_arr[i] for i in result[0]]

    # 출력의 끝을 의미하는 'E' 이후의 글자들을 제거하고 문자열로 만든다.
    end = decoded.index('E')
    translated = ''.join(decoded[:end])

    return translated


In [14]:
print('\n=== 번역 테스트 ===')

print('tree ->', translate('tree'))
print('food ->', translate('food'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('carf ->', translate('carf'))


=== 번역 테스트 ===
tree -> 나무
food -> 음식
love -> 사랑
loev -> 사랑
carf -> 카드
